# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

$$ C = w_M * L_M + w_F * L_F $$

$$ Q = C^\omega * H^(1-\omega) $$

$$ T_M = L_M + H_M $$

$$ T_F = L_F + H_F $$

$$ L_M, H_M, L_F, H_F >= 0 $$

$$ T_M, T_F <= 24 $$

# Setting the basics

In [1]:
%load_ext autoreload
%autoreload 2

from types import SimpleNamespace

import numpy as np
from scipy import optimize

import pandas as pd 
import matplotlib.pyplot as plt

import math as math

from scipy.optimize import minimize, minimize_scalar


In [2]:

import household as model
hsm = model.HouseholdSpecializationModelClass()

model=hsm
par =model.par
sol =model.sol


# Question 1

**Explain how you solve the model**

In [3]:
#Minimum i household.py virker ikke pt, skal der kigges på 
#Det er løst for sigma=1 og alpha=0.5, der skal findes ud af hvordan man løse for alle de værdier opgivet i opgaven
dsol = hsm.solve_discrete()

print(f'HF: {dsol.HF} for alpha and sigma: {hsm.par.alpha, hsm.par.sigma}')
print(f'HM: {dsol.HM} for alpha and sigma: {hsm.par.alpha, hsm.par.sigma}')

print(dsol.HF/dsol.HM)

/Users/josefine/ITP/projects-2023-jc/inauguralproject/household.py:68: RuntimeWarning: invalid value encountered in reciprocal
  utility = np.fmax(Q,1e-8)**(1-par.rho)/(1-par.rho)


HF: 4.5 for alpha and sigma: (0.5, 1.0)
HM: 4.5 for alpha and sigma: (0.5, 1.0)
1.0


# Question 2

Explain your code and procedure

In [4]:
#Test af log (skal ikke bruges)
logHM = math.log(dsol.HM)
logHF = math.log(dsol.HF)
logwF = math.log(par.wF)
logwM = math.log(par.wM)

# log til opgave 2 for H og w
logH = math.log(dsol.HF/dsol.HM)
logw = math.log(par.wM/par.wF)
print(logH)
print(logw)

0.0
0.0


Comments and conclusion:


# Question 3

Explain your code and procedure

Comments and conclusion

# Question 4

Comments and conclusion

# Question 5

Comments and conclusion


In [ ]:
#**Diverse
# Parameters
rho = 2
v = 0.001
epsilon = 1
omega = 0.5
alpha = 0.5
sigma = 1 
wages_m = 1
wages_f = 1
labour_m = 30
labour_f = 20
hours_m = 12
hours_f = 10

consumption = wages_m*labour_m + wages_f*labour_f
print(consumption)

t_m=labour_m+hours_m
t_f=labour_f+hours_f


def h(hour_m, hour_f, alpha, sigma):
    h[sigma = 0] = np.fmin(hours_m,hours_f)
    h[sigma = 1] = hours_m**(1-alpha)*hours_f**(alpha)
    h[simga != 0,1] =(1-alpha)*hours_m**((sigma-1)/sigma)+alpha*hours_f**((sigma-1)/sigma)
    
    return h

q = consumption**omega*h**(1-omega)